In [1]:
import re
import nltk

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import os, re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

In [4]:
print("Upload your .txt lyric files...")
uploaded_files = files.upload()

documents = []
filenames = []
for filename in uploaded_files.keys():
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        documents.append(f.read())
        filenames.append(filename)

Upload your .txt lyric files...


Saving rock Sad.txt to rock Sad.txt
Saving pop sad .txt to pop sad .txt
Saving jazzy sad.txt to jazzy sad.txt
Saving Jazzy Happy.txt to Jazzy Happy.txt
Saving Happy POP.txt to Happy POP.txt
Saving ROCK Happy .txt to ROCK Happy .txt
Saving Jazzy Energetic.txt to Jazzy Energetic.txt
Saving jazzy nostalgic.txt.txt to jazzy nostalgic.txt.txt
Saving pop energetic.txt to pop energetic.txt
Saving POP Nostalgic.txt to POP Nostalgic.txt
Saving Rock Emotional.txt to Rock Emotional.txt
Saving rock energetic.txt to rock energetic.txt
Saving Rock Nostalgic.txt to Rock Nostalgic.txt
Saving pop Emotional.txt to pop Emotional.txt
Saving Jazzy Emotional.txt to Jazzy Emotional.txt


In [5]:
HAPPY = set("""
happy joy joyful joyfulness love lovely sunshine sunny bright smile smiles party dance dancing celebrate celebration
good great amazing awesome best upbeat cherish hug hugs kisses shining shine victory together freedom fun phenomenal
bliss cheerful delight delighted laughter laugh grin grinning optimism optimistic
""".split())

SAD = set("""
sad sorrow pain lonely alone tears crying cry hurt hurting broken breakdown regret regrets mistake mistakes bitter
bittersweet darkness dark cold miss missing goodbye over apart worry worries burden heavy tired blue blues despair
heartbreak heartbroken grief grieving melancholy mourn mourning downcast
""".split())

ENERGETIC = set("""
energy energetic fire electric lightning speed rush power pulse beat beats bass jump move shake wild roar ignite pump
hype turbo bounce sprint adrenaline storm thunder rave mosh slam drive fast furious racing heat hustle tempo
""".split())

NOSTALGIC = set("""
nostalgic nostalgia memory memories remember remembering reminiscent reminisce yesterday yesterdays throwback rewind
olden golden vintage classic hometown homecoming home return again childhood teenage school college summer winter spring fall
decade back then backthen old days
""".split())

EMOTIONAL = set("""
tears tear heart hearts heartfelt feeling feelings emotion emotional fragile tender touch touching soulful deep ache aching
cry crying broken break breaking bleed bleeding longing yearning empathy comfort comforted
""".split())

SENTI_LEX = {
    "Happy": HAPPY,
    "Sad": SAD,
    "Energetic": ENERGETIC,
    "Nostalgic": NOSTALGIC,
    "Emotional": EMOTIONAL,
}

def normalize(text: str):
    return re.findall(r"[a-zA-Z]+(?:'[a-zA-Z]+)?", text.lower())


def mood_label(score: float) -> str:
    return "happy" if score >= 0 else "sad"

def detect_bollywood_hollywood(filename: str, text: str) -> str:
    name = filename.lower()
    if "bollywood" in name: return "bollywood"
    if "holly" in name or "hollywood" in name: return "hollywood"
    if re.search(r"[\u0900-\u097F]", text):  # check Hindi script
        return "bollywood"
    return "hollywood"

In [6]:
def sentiment_scores(text: str):
    words = normalize(text)
    n = max(1, len(words))
    scores = {}
    for label, lex in SENTI_LEX.items():
        count = sum(1 for w in words if w in lex)
        scores[label] = count / n
    return scores

In [7]:
def mood_label(scores: dict) -> str:
    best = max(scores, key=scores.get)
    if all(v == 0 for v in scores.values()):
        return "Emotional"
    return best

In [8]:
GENRE_KEYWORDS = {
    "jazz": set("""
        jazz swing blue blues improv improvisation solo solos sax saxophone trumpet horn bebop cool fusion groove
        syncopation scat quartet trio brushes upright
    """.split()),
    "rock": set("""
        rock guitar riff riffs drum drums amp amps band crowd scream grunge metal punk mosh solo solos distortion
        power chord stadium garage
    """.split()),
    "pop": set("""
        pop radio chorus catchy hook dancefloor chart bubblegum glitter mainstream hit hits single remix
    """.split()),
}

def detect_genre(filename: str, text: str) -> str:
    name = filename.lower()
    if "jazz" in name: return "jazz"
    if "rock" in name: return "rock"
    if "pop"  in name: return "pop"

    words = set(normalize(text))
    # count keyword hits per genre
    hits = {g: sum(1 for w in words if w in kw) for g, kw in GENRE_KEYWORDS.items()}
    # pick the genre with max hits; default to 'pop' if all zero
    best = max(hits, key=hits.get) if hits else "pop"
    if all(v == 0 for v in hits.values()):
        return "pop"
    return best


In [9]:
records = []
for fname, text in zip(filenames, documents):
    scores = sentiment_scores(text)
    label = mood_label(scores)
    records.append({
        "filename": fname,
        "text": text,
        "sentiment_score": scores[label],  # score for the chosen label
        "mood": label,                     # one of: Happy/Sad/Energetic/Nostalgic/Emotional
        "genre": detect_genre(fname, text) # one of: jazz/rock/pop
    })

df = pd.DataFrame(records)
print(df[["filename", "mood", "genre", "sentiment_score"]].head())

          filename       mood genre  sentiment_score
0     rock Sad.txt  Emotional  rock         0.025157
1     pop sad .txt      Happy   pop         0.043689
2    jazzy sad.txt  Emotional  jazz         0.032000
3  Jazzy Happy.txt  Emotional  jazz         0.000000
4    Happy POP.txt  Nostalgic   pop         0.003311


In [10]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["text"].tolist())

def pick_lyrics(theme: str, mood: str, genre: str):
    # strict: match both mood + genre
    mask = (df["mood"].str.lower() == mood.lower()) & (df["genre"].str.lower() == genre.lower())
    candidates = df[mask]

    # fallback 1: match mood only
    if candidates.empty:
        candidates = df[df["mood"].str.lower() == mood.lower()]

    # fallback 2: all docs
    if candidates.empty:
        candidates = df

    user_vec = vectorizer.transform([theme])
    cand_indices = candidates.index.tolist()
    cand_matrix = tfidf_matrix[cand_indices]
    sims = cosine_similarity(user_vec, cand_matrix).flatten()
    best_idx_local = sims.argmax()
    best_row = candidates.iloc[best_idx_local]
    return best_row, float(sims[best_idx_local])

In [11]:
theme = "dance party celebration love"
mood = "Energetic"   # choose from: Happy, Sad, Energetic, Nostalgic, Emotional
genre = "Jazz"       # choose from: Jazz, Rock, Pop (case-insensitive)

row, sim = pick_lyrics(theme, mood, genre)
print("Chosen file:", row["filename"])
print("Mood:", row["mood"], "| Genre:", row["genre"], "| Score:", row["sentiment_score"])
print("Cosine similarity:", sim, "\n")
print("--- Full Lyrics ---\n")
print(row["text"])

Chosen file: pop energetic.txt
Mood: Energetic | Genre: pop | Score: 0.008174386920980926
Cosine similarity: 0.0 

--- Full Lyrics ---

Firework – Katy Perry

Do you ever feel like a plastic bag
Drifting through the wind, wanting to start again?
Do you ever feel, feel so paper thin
Like a house of cards, one blow from cavin' in?
Do you ever feel already buried deep?
Six feet under screams, but no one seems to hear a thing
Do you know that there's still a chance for you?
'Cause there's a spark in you
You just gotta ignite the light
And let it shine
Just own the night
Like the Fourth of July
'Cause baby, you're a firework
Come on, show 'em what you're worth
Make 'em go, "Oh, oh, oh"
As you shoot across the sky
Baby, you're a firework
Come on, let your colors burst
Make 'em go, "Oh, oh, oh"
You're gonna leave 'em all in awe, awe, awe
You don't have to feel like a waste of space
You're original, cannot be replaced
If you only knew what the future holds
After a hurricane comes a rainbow
May